# Part 1

In [1]:
test = ['Sensor at x=2, y=18: closest beacon is at x=-2, y=15',
        'Sensor at x=9, y=16: closest beacon is at x=10, y=16',
        'Sensor at x=13, y=2: closest beacon is at x=15, y=3',
        'Sensor at x=12, y=14: closest beacon is at x=10, y=16',
        'Sensor at x=10, y=20: closest beacon is at x=10, y=16',
        'Sensor at x=14, y=17: closest beacon is at x=10, y=16',
        'Sensor at x=8, y=7: closest beacon is at x=2, y=10',
        'Sensor at x=2, y=0: closest beacon is at x=2, y=10',
        'Sensor at x=0, y=11: closest beacon is at x=2, y=10',
        'Sensor at x=20, y=14: closest beacon is at x=25, y=17',
        'Sensor at x=17, y=20: closest beacon is at x=21, y=22',
        'Sensor at x=16, y=7: closest beacon is at x=15, y=3',
        'Sensor at x=14, y=3: closest beacon is at x=15, y=3',
        'Sensor at x=20, y=1: closest beacon is at x=15, y=3']

In [2]:
def get_sensors(data):
    sensors = {}
    for line in data:
        line = line.strip().split()
        sx = int(line[2][2:-1])
        sy = int(line[3][2:-1])
        bx = int(line[-2][2:-1])
        by = int(line[-1][2:])
        
        manhatten = abs(sx-bx)+abs(sy-by)
        sensors[(sx,sy)] = [(bx,by),manhatten]
    return sensors

def unable_at_y(sensors, y, empty=None):
    if empty is None:
        empty = {}
    for sensor in sensors.keys():
        if abs(sensor[1]-y) > sensors[sensor][1]:
            continue
        dy = abs(y-sensor[1])
        dx = sensors[sensor][1]-dy
        min_x = sensor[0]-dx
        max_x = sensor[0]+dx
        for x in range(min_x, max_x+1):
            if (x,y) not in empty.keys():
                empty[(x,y)] = 1
                
    for sensor in sensors.keys():
        empty.pop(sensors[sensor][0], None)
                
    return empty.keys()

def run_part1(data, y):
    sensors = get_sensors(data)
    number = unable_at_y(sensors, y)
    print('Part 1 result:', len(list(number)))

In [3]:
run_part1(test, 10)

Part 1 result: 26


In [4]:
with open('day15_input.txt', 'r') as f:
    inpt = f.readlines()
    f.close()
run_part1(inpt, 2000000)

Part 1 result: 4886370


# Part 2

In [5]:
def find_gap(sensors, maxi):
    for y in range(0, maxi+1):
        rnge = []
        for sensor in sensors.keys():
            if abs(sensor[1]-y) > sensors[sensor][1]:
                continue
                
            dy = abs(y-sensor[1])
            dx = sensors[sensor][1]-dy
            min_x = sensor[0]-dx
            if min_x > maxi:
                continue
            elif min_x < 0:
                min_x = 0
            max_x = sensor[0]+dx
            if max_x < 0:
                continue
            elif max_x > maxi:
                max_x = maxi
                
            if len(rnge) == 0:
                rnge.append([min_x, max_x])
            else:
                count = 0
                for i in range(0, len(rnge)):
                    #inside rnge[i]
                    if rnge[i][0] <= min_x and rnge[i][1] >= max_x:
                        count += 1
                    #outside rnge[i]
                    elif rnge[i][0] >= min_x and rnge[i][1] <= max_x:
                        rnge[i] = [min_x, max_x]
                        count += 1
                    #left of rnge[i]
                    elif min_x <= rnge[i][0] and rnge[i][0] <= max_x and max_x <= rnge[i][1]:
                        rnge[i][0] = min_x
                        count +=1
                    #right of rnge[i]
                    elif rnge[i][0] <= min_x and min_x <= rnge[i][1] and rnge[i][1] <= max_x:
                        rnge[i][1] = max_x
                        count += 1
                    #right and butting
                    elif rnge[i][1]+1 == min_x:
                        rnge[i][1] = max_x
                        count += 1
                    #left and butting
                    elif rnge[i][0] == max_x+1:
                        rnge[i][0] = min_x
                        count += 1
                if count == 0:
                    rnge.append([min_x,max_x])
            
            if len(rnge) > 1:
                while True:
                    to_del = []
                    len_rnge = len(rnge)
                    for i in range(0, len_rnge-1):
                        if i in to_del:
                            continue
                        for j in range(i+1, len_rnge):
                            if j in to_del:
                                continue
                            #inside rnge[i]
                            if rnge[i][0] <= rnge[j][0] and rnge[i][1] >= rnge[j][1]:
                                to_del.append(j)
                            #outside rnge[i]
                            elif rnge[i][0] >= rnge[j][0] and rnge[i][1] <= rnge[j][1]:
                                rnge[i] = [rnge[j][0], rnge[j][1]]
                                to_del.append(j)
                            #left of rnge[i]
                            elif rnge[j][0] <= rnge[i][0] and rnge[i][0] <= rnge[j][1] and rnge[j][1] <= rnge[i][1]:
                                rnge[i][0] = rnge[j][0]
                                to_del.append(j)
                            #right of rnge[i]
                            elif rnge[i][0] <= rnge[j][0] and rnge[j][0] <= rnge[i][1] and rnge[i][1] <= rnge[j][1]:
                                rnge[i][1] = rnge[j][1]
                                to_del.append(j)
                            #right and butting
                            elif rnge[i][1]+1 == rnge[j][0]:
                                rnge[i][1] = rnge[j][1]
                                to_del.append(j)
                            #left and butting
                            elif rnge[i][0] == rnge[j][1]+1:
                                rnge[i][0] = rnge[j][0]
                                to_del.append(j)
                    if len(to_del) > 0:
                        for i in range(len(to_del)-1, -1, -1):
                            del rnge[to_del[i]]
                    else:
                        break
                        
        if len(rnge) == 2:
            x = min([rnge[0][1], rnge[1][1]])
            return ((x+1)*4000000) + y
        
    return -1

def run_part2(data, maxi):
    sensors = get_sensors(data)
    number = find_gap(sensors, maxi)
    print('Part 2 result:', number)

In [6]:
run_part2(test, 20)

Part 2 result: 56000011


In [7]:
run_part2(inpt, 4000000)

Part 2 result: 11374534948438
